In [1]:
import pandas as pd
from ..gender_detection.names import names_f, names_m

In [2]:
faces_dir = pd.read_parquet('../data/02_intermediate/faces_dir')

In [9]:
faces_dir['gender'] = faces_dir['name'].apply(
    lambda x: 'M' if sum([n in x.split(' ') for n in names_m]) > 0 else ('F' if sum([n in x.split(' ') for n in names_f]) > 0 else 'N') 
)

In [12]:
faces_dir[faces_dir['gender'] != 'N'].to_parquet('../data/03_primary/faces_dir_gender')

In [13]:
faces_vgg = pd.read_parquet('../data/03_primary/faces_vgg_2')

In [17]:
faces_vgg = pd.merge(faces_dir[['gender']],faces_vgg, how='left',left_index=True, right_index=True)

In [22]:
faces_vgg = faces_vgg[faces_vgg['gender']!='N']

In [24]:
names = faces_vgg['name'].unique()

In [26]:
import numpy as np

In [28]:
names = np.random.permutation(names)

In [31]:
train_names = names[:int(0.6*len(names))]
cv_names = names[int(0.6*len(names)):int(0.8*len(names))]
test_names = names[int(0.8*len(names)):]

In [36]:
faces_vgg[faces_vgg['name'].isin(train_names)].drop(['address','number','name'],axis=1).to_parquet('../data/04_model_input/train_gd')
faces_vgg[faces_vgg['name'].isin(cv_names)].drop(['address','number','name'],axis=1).to_parquet('../data/04_model_input/cv_gd')
faces_vgg[faces_vgg['name'].isin(test_names)].drop(['address','number','name'],axis=1).to_parquet('../data/04_model_input/test_gd')